# Houston area: Rooms/Shares

In [29]:
#import get to call a get request on the site
from requests import get

In [30]:
#get the first page of the east bay housing prices
response = get('https://houston.craigslist.org/search/roo') #get rid of those lame-o's that post a housing option without a pic using their filter

print(response.text[:500])

<!DOCTYPE html>
<html class="no-js">
<head>
    
	<meta charset="UTF-8">
	<meta http-equiv="X-UA-Compatible" content="IE=Edge">
	<meta name="viewport" content="width=device-width,initial-scale=1">
	<meta property="og:site_name" content="craigslist">
	<meta name="twitter:card" content="preview">
	<meta property="og:title" content="houston rooms &amp; shares - craigslist">
	<meta name="description" content="houston rooms &amp; shares - craigslist">
	<meta property="og:description" content="houston


In [31]:
from bs4 import BeautifulSoup

In [32]:
#grab the HTML as a BS4 soup object
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [33]:
#get the macro-container for the housing posts
posts = html_soup.find_all('li', class_= 'result-row')                ###
print(type(posts)) #to double check that I got a ResultSet
print(len(posts)) #to double check I got 50 (elements/page)

<class 'bs4.element.ResultSet'>
120


In [42]:
#grab the first post
post_one = posts[1]
post_one

<li class="result-row" data-pid="7331476934">
<a class="result-image gallery" data-ids="3:00X0X_50TVHhRb1ntz_0t20CI,3:00606_aDtW5XCDbRQz_0t20CI,3:00g0g_5hPWVXikZ3wz_0CI0t2,3:00n0n_eoVOcolpf3Wz_0t20CI,3:01515_5WOYs1Ly6hZz_0CI0t2,3:00202_fXZlImfL9Nxz_0CI0t2,3:00a0a_bcR2LPy2j5cz_0t20CI,3:00A0A_8Y77S7cIHmiz_0t20CI,3:00707_9KUFRCcTKVEz_0t20CI" href="https://houston.craigslist.org/roo/d/houston-rooms-in-montrose-garage/7331476934.html">
<span class="result-price">$975</span>
</a>
<div class="result-info">
<span class="icon icon-star" role="button">
<span class="screen-reader-text">favorite this post</span>
</span>
<time class="result-date" datetime="2021-06-23 20:49" title="Wed 23 Jun 08:49:37 PM">Jun 23</time>
<h3 class="result-heading">
<a class="result-title hdrlnk" data-id="7331476934" href="https://houston.craigslist.org/roo/d/houston-rooms-in-montrose-garage/7331476934.html" id="postid_7331476934">$975 - $1,025 2 Rooms in Montrose Garage Apartment - August 1st</a>
</h3>
<span class="re

In [43]:
#grab the price of the first post
post_one_price = post_one.a.text
post_one_price

'\n$975\n'

In [44]:
#grab the time of the post in datetime format to save on cleaning efforts
post_one_time = post_one.find('time', class_= 'result-date')
post_one_datetime = post_one_time['datetime']

#split them and grab each element of the resulting list to get date and time
post_one_datetime = post_one_datetime.split(' ')
post_one_date = post_one_datetime[0]
post_one_date

#element 2 is the time
post_one_time = post_one_datetime[1]

post_one_time

'20:49'

In [45]:
#title is a and that class, link is grabbing the href attribute of that variable
post_one_title = post_one.find('a', class_='result-title hdrlnk')
post_one_link = post_one_title['href']
post_one_link

'https://houston.craigslist.org/roo/d/houston-rooms-in-montrose-garage/7331476934.html'

In [46]:
#easy to grab the post title by taking the text element of the title variable
post_one_title_text = post_one_title.text
post_one_title_text

'$975 - $1,025 2 Rooms in Montrose Garage Apartment - August 1st'

In [47]:
#the neighborhood is grabbed by finding the span class 'result-hood' and pulling the text element from that
post_one_hood = post_one.find('span', class_= 'result-hood').text
post_one_hood

' (Montrose)'

In [48]:
#build out the loop
from time import sleep
from random import randint
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np

#find the total number of posts to find the limit of the pagination
results_num = html_soup.find('div', class_= 'search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)

pages = np.arange(0, results_total, 120)

iterations = 0

post_timing = []
post_hoods = []
post_title_texts = []
post_links = []
post_prices = []

for page in pages:
    
    #get request
    response = get("https://sfbay.craigslist.org/search/eby/roo?" 
                   + "s=" 
                   + str(page) 
                   + "&hasPic=1"
                   + "&availabilityMode=0")

    sleep(randint(1,5))
     
    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
        
    #define the html text
    page_html = BeautifulSoup(response.text, 'html.parser')
    
    #define the posts
    posts = html_soup.find_all('li', class_= 'result-row')
        
    #extract data item-wise
    for post in posts:

        if post.find('span', class_ = 'result-hood') is not None:

            #posting date
            #grab the datetime element 0 for date and 1 for time
            post_datetime = post.find('time', class_= 'result-date')['datetime']
            post_timing.append(post_datetime)

            #neighborhoods
            post_hood = post.find('span', class_= 'result-hood').text
            post_hoods.append(post_hood)

            #title text
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_title_texts.append(post_title_text)

            #post link
            post_link = post_title['href']
            post_links.append(post_link)

            post_price = post.a.text
            post_prices.append(post_price)


    iterations += 1
    print("Finished iteration: " + str(iterations))

Finished iteration: 1
Finished iteration: 2
Finished iteration: 3
Finished iteration: 4
Finished iteration: 5
Finished iteration: 6
Finished iteration: 7


In [41]:
import pandas as pd

eb_rooms = pd.DataFrame({'posted': post_timing,
                       'neighborhood': post_hoods,
                       'post title': post_title_texts,
                       'URL': post_links,
                       'price': post_prices})
print(eb_rooms.info())
eb_rooms.tail(1000)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   posted        588 non-null    object
 1   neighborhood  588 non-null    object
 2   post title    588 non-null    object
 3   URL           588 non-null    object
 4   price         588 non-null    object
dtypes: object(5)
memory usage: 23.1+ KB
None


,posted,neighborhood,post title,URL,price
0,2021-06-23 20:51,(Houston),GL Yng Male roommate wanted,https://houston.craigslist.org/roo/d/houston-g...,
1,2021-06-23 20:49,(Montrose),"$975 - $1,025 2 Rooms in Montrose Garage Apart...",https://houston.craigslist.org/roo/d/houston-r...,\n$975\n
2,2021-06-23 20:36,(CANDLELIGHT FOREST WEST),Room For Rent⭐⭐⭐,https://houston.craigslist.org/roo/d/houston-r...,\n$625\n
3,2021-06-23 19:41,(Sawyer Heights / Arts District),ALL BILLS INCL*** FURNISHED MASTER BEDROOM / W...,https://houston.craigslist.org/roo/d/all-bills...,"\n$1,250\n"
4,2021-06-23 19:36,(Katy - Fulshear / Tamarron),Brand New Room Upstairs For Rent $600,https://houston.craigslist.org/roo/d/katy-bran...,\n$600\n
...,...,...,...,...,...
583,2021-06-22 21:08,(houston),Renting a room for a single female living room,https://houston.craigslist.org/roo/d/houston-r...,\n$650\n
584,2021-06-22 20:55,(Downtown),Penthouse sweet,https://houston.craigslist.org/roo/d/houston-p...,\n$0\n
585,2021-06-22 20:40,(Midtown),Midtown furnished room with private bathroom,https://houston.craigslist.org/roo/d/houston-m...,\n$990\n
586,2021-06-22 20:10,(The Woodlands/Spring),Furnished room for rent in the heart of The Wo...,https://houston.craigslist.org/roo/d/spring-fu...,


In [57]:
eb_rooms.to_csv('HoustonROOMSharing.csv',  encoding='utf-8', header='true')